Requirements 

>>  just imagine you are working in a project which is not using Delta Lake , But they have the files in parquet format 

>> Now you got a new requirement where you want to convert all these parquet files to delta.


In [0]:
dbutils.fs.ls('/mnt/DeltaLake/Test/only_parquet')

[FileInfo(path='dbfs:/mnt/DeltaLake/Test/only_parquet/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1709104999000),
 FileInfo(path='dbfs:/mnt/DeltaLake/Test/only_parquet/_committed_5733939502324421826', name='_committed_5733939502324421826', size=236, modificationTime=1709104999000),
 FileInfo(path='dbfs:/mnt/DeltaLake/Test/only_parquet/_committed_6143306704378934151', name='_committed_6143306704378934151', size=125, modificationTime=1709104875000),
 FileInfo(path='dbfs:/mnt/DeltaLake/Test/only_parquet/_started_5733939502324421826', name='_started_5733939502324421826', size=0, modificationTime=1709104999000),
 FileInfo(path='dbfs:/mnt/DeltaLake/Test/only_parquet/_started_6143306704378934151', name='_started_6143306704378934151', size=0, modificationTime=1709104875000),
 FileInfo(path='dbfs:/mnt/DeltaLake/Test/only_parquet/part-00000-tid-5733939502324421826-d3eba2d8-2e0b-4978-b435-5a747c3a78db-2936-1.c000.snappy.parquet', name='part-00000-tid-5733939502324421826-d3eba2d8-2e0b-497

In [0]:
filepath = 'dbfs:/mnt/DeltaLake/Test/SchemaManagementDelta.csv'

df = spark.read.format('csv').option('Header', 'True').option('inferSchema', 'True').load(filepath)

In [0]:
## write to parquet files 

op = 'dbfs:/mnt/DeltaLake/Test'
df.write.format('parquet').save(f'{op}/only_parquet')

In [0]:
## Do some transformation on top of df 

dfNew = df.withColumnRenamed('Line_Number', 'LNo')

In [0]:
dfNew.write.format('parquet').mode('overwrite').save(f'{op}/only_parquet')

## Now conver that parquet file to delta format and overwrite to same location 


In [0]:
%sql 

convert to delta parquet.`dbfs:/mnt/DeltaLake/Test/only_parquet`

In [0]:
%sql
DESCRIBE History delta.`dbfs:/mnt/DeltaLake/Test/only_parquet`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2024-02-28T07:26:10Z,4700893722167418,lokeswar.valluru@xyenta.com,CONVERT,"Map(numFiles -> 1, partitionedBy -> [], collectStats -> true, sourceFormat -> parquet)",null,List(192613546049837),1227-053858-m1tsqg19,-1,Serializable,false,Map(numConvertedFiles -> 1),null,Databricks-Runtime/13.3.x-photon-scala2.12


>>  the reason why I done Rename Transaction before converting this into the data lake or before overwriting this, To understand Can we get the previous version of this line number or not?.

>. It's not possible to Do versioning. Beacuse versioning will come into picture when we do transformations on delta table ot delta files only 

##Optimize is used to compacting Small Files using Optimize command Optimize by ZORDER BY Column_name (optimize by Indexing)

## Also used to get rid of the InActive parquet files at time multiple operations are done 

In [0]:
%sql

-- OPTIMIZE my_table ZORDER BY col_name ;

-- OPTIMIZE my_table;